In [12]:
import pandas as pd
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import multiprocessing


In [7]:

#csv_path = ".../hotel-reviews.csv"


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
path = '/content/drive/MyDrive/ML/Domain Word Embedding/Data/more_sales_data.xlsx'

import pandas as pd
df = pd.read_excel(path)
print(df)


                  Domain  category
0                oet.com      10.0
1                gbr.com      10.0
2                yae.com      10.0
3               gian.com      10.0
4      bitcoinwisdom.com      10.0
...                  ...       ...
18951    pillowsocks.com       3.0
18952           gmhr.com       3.0
18953       cerealty.com       3.0
18954  stoneandcloth.com       3.0
18955       ladyslim.com       3.0

[18956 rows x 2 columns]


In [16]:

#df = pd.read_csv(csv_path)

def clean_data(text):
    text = re.sub(r'[^ \nA-Za-z0-9À-ÖØ-öø-ÿ/]+', '', text)
    text = re.sub(r'[\\/×\^\]\[÷]', '', text)
    return text

def change_lower(text):
    text = text.lower()
    return text

stopwords_list = stopwords.words("english")

def remover(text):
    text_tokens = text.split(" ")
    final_list = [word for word in text_tokens if not word in stopwords_list]
    text = ' '.join(final_list)
    return text

def get_w2vdf(df):
    w2v_df = pd.DataFrame(df["sentences"]).values.tolist()
    for i in range(len(w2v_df)):
        w2v_df[i] = w2v_df[i][0].split(" ")
    return w2v_df

def train_w2v(w2v_df):
    cores = multiprocessing.cpu_count()
    w2v_model = Word2Vec(min_count=4,
                         window=4,
                         size=300, 
                         alpha=0.03, 
                         min_alpha=0.0007, 
                         sg = 1,
                         workers=cores-1)
    
    w2v_model.build_vocab(w2v_df, progress_per=10000)
    w2v_model.train(w2v_df, total_examples=w2v_model.corpus_count, epochs=100, report_delay=1)
    return w2v_model

df[["sentences"]] = df[["sentences"]].astype(str)
df["sentences"] = df["sentences"].apply(change_lower)
df["sentences"] = df["sentences"].apply(clean_data)
df["sentences"] = df["sentences"].apply(remover)

w2v_df = get_w2vdf(df)
w2v_model = train_w2v(w2v_df)

KeyError: ignored